In [353]:
import pandas as pd
from collections import Counter
import numpy as np
import sys

Загрузим данные

In [356]:
data = pd.read_table("Log", encoding='utf-8')

In [357]:
data["request"].head()

0    https://yandex.ru/search/?text=порно видео сек...
1    https://yandex.ru/search/?text=sad story скача...
2    https://yandex.ru/search/?text=ольгино баня&fl...
3    https://yandex.ru/search/?text=фирменный магаз...
4          https://yandex.ru/search/?text=порно&rl=313
Name: request, dtype: object

Выделим запросы без url

In [358]:
url_part = "https://yandex.ru/search/?text="
data["reduced_request"] = data["request"].apply(lambda x: x[len(url_part):])

In [359]:
data["reduced_request"].head()

0                           порно видео секс за деньги
1    sad story скачать бесплатно mp3 в хорошем каче...
2                              ольгино баня&flag=l6707
3      фирменный магазин марко в г минске скидки акции
4                                         порно&rl=313
Name: reduced_request, dtype: object

Теперь избавимся от лишних символов в конце запроса

In [360]:
data["reduced_request"] = data["reduced_request"].apply(lambda x: x[:x.find("&")] if "&" in x else x)

In [361]:
data["reduced_request"].head()

0                           порно видео секс за деньги
1    sad story скачать бесплатно mp3 в хорошем каче...
2                                         ольгино баня
3      фирменный магазин марко в г минске скидки акции
4                                                порно
Name: reduced_request, dtype: object

Приведем все строки к нижнему регистру

In [362]:
data["reduced_request"] = data["reduced_request"].apply(lambda x: x.lower())

Проведем "наивную" классификацию запросов, связанных с телевидением. Для этого составим словари с ключевыми словами
direct - прямые запросы,
channels - тв каналы,
celeb - тв знаменитости,
prog - тв передачи,
connect - подключение тв

In [363]:
request_dict = {"direct": ["телевидение", "телевидении", "телевидения", "телевидению",
                           " tv ", "-tv", "tv-", " тв ", "-тв", "тв-", "ntktdbltybt"],
               "channels": ["телеканал", "канал", "rfyfk", " стс ", " орт ", " нтв ", "рен тв", " тнт ",
                           " стс", " орт", " нтв", " рентв", " тнт","орт ", "нтв ", "тнт ", "стс ", " ртр ", " ртр", "ртр "
                            "матч тв", "спорт тв", "россия 1", "россия1", "музтв", "муз тв"],
               "celeb": ["максим дахненко", "комаров дмитрий", "елена степаненко", "яна рудковская", "ольга бузова",
                         "дмитрий нагиев", "светлаков", "светлаков", "малахов", "батрутдинов", "канделаки", "ургант"],
               "prog": ["камеди", "comedy", "воскресный вечер", "вести недели", "давай поженимся", "топ гир", "пусть говорят",
                       "ревизорро", "битва экстасенсов", "дом 2", "вечерний ургант", "жди меня", "танцы со звездами",
                        "орел и решка"],
               "connect": ["спутниковое", "кабельное", "эфир"]}

In [364]:
request_class_dict={"direct": [], "channels": [], "celeb": [], "prog": [], "connect": []}
for key in request_dict.keys():
    for value in request_dict[key]:
        request_class_dict[key].append(data[data["reduced_request"].str.contains(value)]["reduced_request"])

In [365]:
request_class_dict["direct"][0][:5]

15672                               телевидение ростелеком
15859         интернет и телевидение от мтс личный кабинет
30756    1канал телевидение прямой эфир сейчас новосибирск
34630          смотреть бесплатно телевидение для взрослых
43835                                          телевидение
Name: reduced_request, dtype: object

Теперь оценим долю запросов, связанных с телевидением в общем потоке. Учтем, что запросы в разных классах могут повторяться

In [366]:
tv_queries_list = []
for key in request_class_dict.keys():
    for value in request_class_dict[key]:
        tv_queries_list += value.tolist()

In [367]:
tv_queries_list[-5:]

['матч тв прямой эфир',
 'футбол барселона сегодня прямой эфир смотреть онлайн бесплатно сейчас',
 'матч тв прямой эфир',
 'матч тв прямой эфир',
 'телеканал че смотреть онлайн прямой эфир бесплатно']

In [368]:
#длина списка запросов с повторениями
print(len(tv_queries_list))
#длина списка запросов без повторений
unique_tv_queries = len(np.unique(tv_queries_list))
print(unique_tv_queries)

16598
9701


Доля запросов, связанных с телевидением, в общем потоке запросов, %

In [354]:
print("Доля запросов, связанных с телевидением, в общем потоке запросов, %", float(unique_tv_queries)/data.shape[0]*100)

Доля запросов, связанных с телевидением, в общем потоке запросов, % 0.7274824278610547


Оценим теперь долю каждого класса в запросах, связанных с телевидением

In [355]:
for key in request_class_dict.keys():
    class_queries_list = []
    for value in request_class_dict[key]:
        class_queries_list += value.tolist()
    print("Доля "+key, float(len(np.unique(class_queries_list)))/unique_tv_queries*100,"%")

Доля direct 14.132563653231625 %
Доля channels 67.07555922069889 %
Доля celeb 2.711060715390166 %
Доля prog 11.400886506545717 %
Доля connect 13.802700752499742 %
